<a href="https://colab.research.google.com/github/GunhyungKim/Blockboxs/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install --upgrade tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 3.2MB 34.3MB/s 
     |████████████████████████████████| 368kB 67.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [ ]:
import cv2
from tensorflow import keras
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
import glob, os
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


In [ ]:
def calculate(TP, TN, FP, FN):
  if TP==0 or TN == 0:
    accuracy, precision, recall = 0, 0, 0
  else:
    accuracy, precision, recall = (TP+TN)/(TP+TN+FP+FN), TP/(TP+FP), TP/(TP+FN)
  return accuracy, precision, recall

In [ ]:
model = load_model('/content/drive/My Drive/Colab Notebooks/new_model.h5')

In [ ]:
threshold = 0.76

err_file = []
err_acc = []

#no_accident
files = glob.glob(os.path.join('/content/drive/My Drive/Colab Notebooks/0818data/test/0', "*"))
files.sort()
c=0

y=[]
for filename in files:
  # print(filename)
  img = cv2.imread(filename)
  img = cv2.resize(img,(224,224))
  img = np.float32(img / 255)
  img = np.reshape(img,[1,224,224,3])

  pred = model.predict(img)

  # print(filename, pred,end=' : ')
  y.append(pred[0][1])
  if threshold < pred[0][1]:
    # print('1')
    c=c+1
    err_file.append(filename)
    err_acc.append(pred[0][1])
  #else:
    # print('0')
  # print(pred)

FP = c
TN = len(files)-c
print('FP : %d 비사고를 사고로' %(FP))
print('TN : ', TN)


files = glob.glob(os.path.join('/content/drive/My Drive/Colab Notebooks/0818data/test/1', "*"))
files.sort()
c=0

y=[]
for filename in files:
  # print(filename)
  img = cv2.imread(filename)
  img = cv2.resize(img,(224,224))
  img = np.float32(img / 255)
  img = np.reshape(img,[1,224,224,3])

  pred = model.predict(img)

  # print(filename, pred,end=' : ')
  y.append(pred[0][1])
  if threshold < pred[0][1]:
    # print('1')
    c=c+1
  else:
    # print('0')
    err_file.append(filename)
    err_acc.append(pred[0][1])
  # print(pred)

TP = c
FN = len(files)-c
print('TP : ' , TP)
print('FN : %d 사고를 비사고로' %(FN))

acc, pre, rec = calculate(TP, TN, FP, FN)
print("acc : %f, pre : %f, rec : %f"%(acc, pre, rec))

FP : 4 비사고를 사고로
TN :  46
TP :  42
FN : 8 사고를 비사고로
acc : 0.880000, pre : 0.913043, rec : 0.840000


In [ ]:
for i in range(0,12):
  print(err_file[i], err_acc[i])

/content/drive/My Drive/Colab Notebooks/0818data/test/0/0056.png 0.94817483
/content/drive/My Drive/Colab Notebooks/0818data/test/0/0112.png 0.95300853
/content/drive/My Drive/Colab Notebooks/0818data/test/0/0114.png 0.98846394
/content/drive/My Drive/Colab Notebooks/0818data/test/0/0152.png 0.99331456
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0055.png 0.001375644
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0091.png 0.22011998
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0094.png 0.03548765
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0111.png 0.13326935
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0153.png 0.00012356152
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0158.png 2.6171041e-05
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0172.png 0.15826415
/content/drive/My Drive/Colab Notebooks/0818data/test/1/0176.png 0.05550016
